In [5]:
import numpy as np

p_dist = [0.1, 0.2, 0.3, 0.4]

info_content = lambda p: np.log2(1/p)
entropy = lambda p_dist: np.sum([p*np.log2(1/p) for p in p_dist])


In [8]:
info_content(p_dist[0])

3.321928094887362

In [9]:
entropy(p_dist)

1.8464393446710154

In [514]:
class TreeNode:
    def __init__(self, data, prob):
        self.data = data
        self.prob = prob
        self.leftChild = None
        self.rightChild = None
        self.treeString = ''
        self.depth = None
        self.leaf = True
    
    def print_leaves(self, level=0):
        self.depth = level
        self.treeString = 'prob:' +str(self.prob) + '--- data:' +str(self.data) +'--- level:' + str(level) + \
        '--- leaf: '+str(self.leaf) +'--->\n'
        if self.rightChild:
            self.treeString += self.rightChild.print_leaves(level+1)
        if self.leftChild:
            self.treeString += self.leftChild.print_leaves(level+1)
        return self.treeString
        

In [515]:
root = TreeNode(0, 0.1)
lhs_child = TreeNode(0, 0.2)
rhs_child = TreeNode(0, 0.3)
root.leftChild = lhs_child
root.rightChild = rhs_child

grand_children = [TreeNode(0, 1/(i+1)) for i in range(4)]
lhs_child.leftChild = grand_children[0]
lhs_child.rightChild = grand_children[1]
rhs_child.leftChild = grand_children[2]
rhs_child.rightChild = grand_children[3]

In [516]:
x = root.print_leaves()
print(x)

prob:0.1--- data:0--- level:0--- leaf: True--->
prob:0.3--- data:0--- level:1--- leaf: True--->
prob:0.25--- data:0--- level:2--- leaf: True--->
prob:0.3333333333333333--- data:0--- level:2--- leaf: True--->
prob:0.2--- data:0--- level:1--- leaf: True--->
prob:0.5--- data:0--- level:2--- leaf: True--->
prob:1.0--- data:0--- level:2--- leaf: True--->



In [451]:
def lhs_1_rhs_0(node, max_level, level=0):
    if node.rightChild is None:
        node.data = 1
        print('did somthing RHS')
    else:
        print('going lower', level)
        lhs_1_rhs_0(node.rightChild, 3, level+1)
        lhs_1_rhs_0(node.leftChild, 3, level+1)
        return 
            

In [517]:
# TODO: build a class with encode and decode methods for the huffman tree
# TODO: try out the ipython UI plugins to make the tree interactive and visualise it

# Implement Huffmans algorithm to find optimal symbol code given an ensemble X w.p P

class HuffmanAlgo:
    
    def __init__(self, P):
        self.P = P
        self.root = None
        self.tree_depth = None
        self.encoder = None
        
    def create_tree(self):
        # create tree from the leaves, starting with the lowest prob combo
        P_sorted = np.sort(self.P)
        P_sorted = [TreeNode(0, prob) for prob in P_sorted]
        leaf_node = P_sorted[0]
        while len(P_sorted) > 1:
            p_0, p_1 = P_sorted[0].prob, P_sorted[1].prob
            p_0_1 = p_0 + p_1
            p_0_1_node = TreeNode(0, p_0_1)
            p_0_1_node.leftChild = P_sorted[0]
            p_0_1_node.rightChild = P_sorted[1]
            p_0_1_node.rightChild.data = 1
            p_0_1_node.leaf = False

#             latest_treeString = p_0_1_node.print_leaves()
#             print(latest_treeString, 'with root prob', p_0_1)

            P_sorted = P_sorted[2:]

            # need to get sorted list of nodes by prob
            P_sorted_plus_new_node = np.concatenate((P_sorted, [p_0_1_node]), axis=0)
            P_sorted = self._sort_nodes_by_prob(P_sorted_plus_new_node)

        # put 1s and 0s at each child node
        self.root = p_0_1_node
        print(self.print_tree())
        self.tree_depth = leaf_node.depth
        
        return 'Tree Created Of Depth: {}'.format(self.tree_depth)
    
    def _sort_nodes_by_prob(self, node_list):
        idx_prob_lookup = {}
        prob_arr = []
        for idx, node in enumerate(node_list):
            idx_prob_lookup[node.prob] = idx
            prob_arr.append(node.prob)
        prob_arr = np.sort(prob_arr)

        sorted_nodes = []
        for prob in prob_arr:
            idx = idx_prob_lookup[prob]
            sorted_nodes.append(node_list[idx])
        return np.array(sorted_nodes)

    
    def print_tree(self):
        
        latest_treeString = self.root.print_leaves()
        return latest_treeString
    
    def encode(self, X):
        
        encoded_X_dict = {}
        encoded_X_path = ''
        encoded_X_dict[leave_name_letter] = encoded_X_path
        # get all paths to target leaves of tree and append data to encoded_X_dict
        
        self.encoder = encoded_X_dict
        return 'Completed encoder'
            

In [518]:
algoInst = HuffmanAlgo(p_dist)
print(algoInst.create_tree())

prob:1.0--- data:0--- level:0--- leaf: False--->
prob:0.6000000000000001--- data:1--- level:1--- leaf: False--->
prob:0.30000000000000004--- data:1--- level:2--- leaf: False--->
prob:0.2--- data:1--- level:3--- leaf: True--->
prob:0.1--- data:0--- level:3--- leaf: True--->
prob:0.3--- data:0--- level:2--- leaf: True--->
prob:0.4--- data:0--- level:1--- leaf: True--->

Tree Created Of Depth: 3


In [501]:
# Assume alphabet is a, b, c, d and prob dist = p_dist defined above
test_X = 'aabcddd'



In [260]:
rcROOT = tree[-1].leftChild
print(tree)
print(rcROOT.prob)
print(rcROOT.rightChild)

[<__main__.TreeNode object at 0x1131f4340>, <__main__.TreeNode object at 0x1131f4a90>, <__main__.TreeNode object at 0x1131f4250>, <__main__.TreeNode object at 0x1131f43a0>, <__main__.TreeNode object at 0x1131f4100>, <__main__.TreeNode object at 0x113211ca0>, <__main__.TreeNode object at 0x113211b50>]
0.1
None


In [3]:
% jupyter nbextension enable --py widgetsnbextension

UsageError: Line magic function `%` not found.


In [2]:
# Implement arithmetic coding

In [10]:
help(np.sort)

Help on function sort in module numpy:

sort(a, axis=-1, kind=None, order=None)
    Return a sorted copy of an array.
    
    Parameters
    ----------
    a : array_like
        Array to be sorted.
    axis : int or None, optional
        Axis along which to sort. If None, the array is flattened before
        sorting. The default is -1, which sorts along the last axis.
    kind : {'quicksort', 'mergesort', 'heapsort', 'stable'}, optional
        Sorting algorithm. The default is 'quicksort'. Note that both 'stable'
        and 'mergesort' use timsort or radix sort under the covers and, in general,
        the actual implementation will vary with data type. The 'mergesort' option
        is retained for backwards compatibility.
    
        .. versionchanged:: 1.15.0.
           The 'stable' option was added.
    
    order : str or list of str, optional
        When `a` is an array with fields defined, this argument specifies
        which fields to compare first, second, etc.  A si